In [64]:
%matplotlib inline

import numpy as np
import pandas as pd

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from pydub import AudioSegment
from pydub.utils import make_chunks

In [65]:
data_dir = './sounds'
audio_files = glob(data_dir + '/*')

In [66]:
len(audio_files)

2

In [67]:
for j in range(len(audio_files)):
    myaudio = AudioSegment.from_file("./sounds/{}".format(os.listdir('./sounds/')[j])) 
    chunk_length_ms = 10000 # pydub calculates in millisec
    chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec

    #Export all of the individual chunks as wav files

    for i, chunk in enumerate(chunks):
        chunk_name = "{}{}.wav".format(os.listdir('./sounds/')[j], i)
        print("exporting", chunk_name)
        chunk.export(chunk_name, format="mp3")

exporting cellphone.mp30.wav
exporting cellphone.mp31.wav
exporting cellphone.mp32.wav
exporting cellphone.mp33.wav
exporting cellphone.mp34.wav
exporting cellphone.mp35.wav
exporting cellphone.mp36.wav
exporting cellphone.mp37.wav
exporting cellphone.mp38.wav
exporting cellphone.mp39.wav
exporting cellphone.mp310.wav
exporting cellphone.mp311.wav
exporting cellphone.mp312.wav
exporting cellphone.mp313.wav
exporting cellphone.mp314.wav
exporting cellphone.mp315.wav
exporting cellphone.mp316.wav
exporting cellphone.mp317.wav
exporting cellphone.mp318.wav
exporting cellphone.mp319.wav
exporting cellphone.mp320.wav
exporting cellphone.mp321.wav
exporting cellphone.mp322.wav
exporting cellphone.mp323.wav
exporting cellphone.mp324.wav
exporting cellphone.mp325.wav
exporting cellphone.mp326.wav
exporting cellphone.mp327.wav
exporting cellphone.mp328.wav
exporting cellphone.mp329.wav
exporting cellphone.mp330.wav
exporting cellphone.mp331.wav
exporting cellphone.mp332.wav
exporting cellphone.

In [70]:
data_dir = './sounds2'
audio_files = glob(data_dir + '/*.wav')

In [71]:
len(audio_files)

152

In [73]:
data = pd.DataFrame(columns = ['name','mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse'])

In [74]:
lista = []
for i in range(len(audio_files)):
    lista.append(os.listdir('./sounds2/')[i])
data.name = lista

In [75]:
lista2 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    mfcc_aux = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    mfcc = np.ndarray.mean(mfcc_aux)
    lista2.append(mfcc)
data.mfcc = lista2

In [76]:
lista3 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    scem = np.ndarray.mean(cent)
    lista3.append(scem)
data.scem = lista3

In [77]:
lista4 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    S = np.abs(librosa.stft(y))
    contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
    scom = np.ndarray.mean(contrast)
    lista4.append(scom)
data.scom = lista4

In [78]:
lista5 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    srom = np.ndarray.mean(rolloff)
    lista5.append(srom)
data.srom = lista5

In [79]:
lista6 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    sbwm = np.ndarray.mean(spec_bw)
    lista6.append(sbwm)
data.sbwm = lista6

In [80]:
lista7 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    hop_length = 512
    oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)
    tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sr, hop_length=hop_length)

    # Compute global onset autocorrelation
    ac_global = librosa.autocorrelate(oenv, max_size=tempogram.shape[0])
    ac_global = librosa.util.normalize(ac_global)

    # Estimate the global tempo for display purposes
    tempo = librosa.beat.tempo(onset_envelope=oenv, sr=sr, hop_length=hop_length)[0]
    lista7.append(tempo)
data.tempo = lista7

In [81]:
lista8 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    rmse_aux = librosa.feature.rms(y=y)
    rmse = np.ndarray.mean(rmse_aux)
    lista8.append(rmse)
data.rmse = lista8

In [82]:
data.shape

(152, 8)

In [83]:
data.dtypes

name      object
mfcc     float64
scem     float64
scom     float64
srom     float64
sbwm     float64
tempo    float64
rmse     float64
dtype: object

In [84]:
momento = []

for i in data['name']:
    if 'ducha' in i:
        momento.append('Ducha')
    elif 'cena' in i:
        momento.append('Cena')
    elif 'washing' in i:
        momento.append('Lavadora')
    elif 'vacuum' in i:
        momento.append('Aspiradora')
    elif 'shaver' in i:
        momento.append('Afeitadora')
    elif 'hairdryer' in i:
        momento.append('Secador_pelo')
    elif 'airconditioner' in i:
        momento.append('Aire_acondicionado')
    elif 'cellphone' in i:
        momento.append('Telefono')
    else:
        momento.append('Otro')
        
data['momento_aux'] = momento
#data['momento'] = 'Otro'
data.drop(['name'], axis=1, inplace=True)

In [85]:
data.momento_aux.value_counts()

Telefono    152
Name: momento_aux, dtype: int64

In [86]:
data.to_csv('data_momento_aux_cellphone.csv', index=False)